In [1]:
#include <type_traits>
#include <mutex>
#include <limits>
#include <initializer_list>
#include <vector>
#include <numbers>
#include <array>
#include <cmath>

## 6.1 constexpr variable

In [2]:
constexpr double PI = 3.1415926535;
constexpr auto area = PI * 2.0 * 2.0;

In [3]:
template<typename T> 
constexpr bool is_integral_v = std::is_integral_v<T>;

In [4]:
static_assert( is_integral_v<int> );

In [5]:
static_assert( not is_integral_v<void>);

In [6]:
template<char c>
constexpr bool is_digit = (c >= '0' and c <= '9');

In [7]:
static_assert(is_digit<'0'>);

In [8]:
static_assert(not is_digit<'x'>);

## 6.2 constinit

In [9]:
// constinit std::mutex g_mtx;

constinit is not supported Cling with gcc 12.2.0 on Debian.

## 6.3 fold expression

In [10]:
template<size_t...Is>
constexpr size_t rsum = (Is + ... + 0);
template<size_t...Is>
constexpr size_t lsum = (0 + ... + Is);

In [11]:
lsum<100>

(const unsigned long) 100


In [12]:
lsum<100, 1>

(const unsigned long) 101


In [13]:
rsum<1, 100>

(const unsigned long) 101


In [14]:
template<size_t...Is>
constexpr int rsub = ( Is - ... - 0);
template<size_t...Is>
constexpr int lsub = (0 - ... - Is);

In [15]:
rsub<100>

(const int) 100


In [16]:
lsub<100>

(const int) -100


In [17]:
rsub<100, 10>

(const int) 90


In [18]:
lsub<100, 10>

(const int) -110


## 6.4 constexpr function

In [19]:
constexpr int min(std::initializer_list<int> xs){
    int low = std::numeric_limits<int>::max();
    for(auto x : xs){
        if(x < low){
            low = x;
        }
    }
    return low;
}

In [20]:
min({1,1,3,4});

(int) 1


It seems cling does not support `consteval`:
```C++
consteval int min_eval(std::initializer_list<int> xs){
    int low = std::numeric_limits<int>::max();
    for(auto x : xs){
        if(x < low){
            low = x;
        }
    }
    return low;
}
```

Compilation time allocated memory should be freed before the end of compilation.

```C++
#include <vector>
#include <array>
#include <iostream>

constexpr std::vector<int> sievePrime(int n){
    std::vector<bool> marked(n+1, true);
    for(int p = 2; p * p < n; ++p){
        if(marked[p]){
            for(int i = p * p; i <= n; i += p){
                marked[i] = false;
            }
        }
    }
    
    std::vector<int> result;
    for(int p = 2; p < n; ++p){
        if(marked[p]){
            result.push_back(p);
        }
    }
    return result;
}

constexpr std::size_t primeCount(int n){
    return sievePrime(n).size();
}

template<int n>
consteval auto savePrimeToArray(){
    std::array<int, primeCount(n)> result;
    auto primes = sievePrime(n);
    std::copy(primes.begin(), primes.end(), result.begin());
    return result;
}

int main(void){
    static_assert(primeCount(100) == 25);
    auto primes = savePrimeToArray<100>();
    for(auto e: primes){
        std::cout << e << std::endl;
    }
}
```

In [21]:
struct Shape{
    virtual ~Shape() = default;
    virtual double getArea() const = 0;
};

In [22]:
struct Circle : Shape{
    constexpr Circle(double r) : r_(r){}
    constexpr double getArea() const override{
        return std::numbers::pi * r_ * r_;
    }
private:
    double r_;
}

In [23]:
struct Rectangle : Shape{
    constexpr Rectangle(double l, double w) : l_(l), w_(w){
    
    }
    constexpr double getArea() const override{
        return l_ * w_;
    }
private:
    double l_;
    double w_;
}

In [24]:
constexpr double testSubType(){
    std::array<Shape*, 2> shapes{
        new Circle(10), new Rectangle(3, 5)
    };
    double sum = 0.0;
    for(auto s : shapes){
        sum += s->getArea();
        delete s;
    }
    return sum;
}

In [25]:
testSubType()

(double) 329.15927


In [26]:
constexpr double testDynamicCast(){
    std::array<Shape*, 2> shapes{
        new Circle(10), new Rectangle(3, 5)
    };
    double sum = 0.0;
    for(auto s : shapes){
        if(auto rec = dynamic_cast<Rectangle*>(s)){
            sum += rec->getArea();
        }
        delete s;
    }
    return sum;
}

In [27]:
testDynamicCast()

(double) 15.000000


Using example from cpp reference:

In [28]:
constexpr double power(double b, int x)
{
    if (std::is_constant_evaluated() && !(b == 0.0 && x < 0))
    {
        // A constant-evaluation context: Use a constexpr-friendly algorithm.
        if (x == 0)
            return 1.0;
        double r {1.0};
        double p {x > 0 ? b : 1.0 / b};
        for (auto u = unsigned(x > 0 ? x : -x); u != 0; u /= 2)
        {
            if (u & 1)
                r *= p;
            p *= p;
        }
        return r;
    }
    else
    {
        // Let the code generator figure it out.
        return std::pow(b, double(x));
    }
}

In [29]:
// A constant-expression context
constexpr double kilo = power(10.0, 3);
int n = 3;

In [30]:
kilo

(const double) 1000.0000


In [31]:
// Not a constant expression, because n cannot be converted to an rvalue
// in a constant-expression context
// Equivalent to std::pow(10.0, double(n))
double mucho = power(10.0, n);

In [32]:
mucho

(double) 1000.0000


In [33]:
static_assert(std::is_constant_evaluated())

In [34]:
int y = 0;

In [35]:
const int a = std::is_constant_evaluated() ? y : 1;

In [36]:
a

(const int) 1


In [37]:
const int b = std::is_constant_evaluated() ? 2 : y;

In [38]:
b

(const int) 2


Collatz Conjecture:
$ f(n) = n / 2 $ if n is even, $ f(n) = 3 n + 1 $ if n is odd, for n > 0 and n is integral, f(n) converge to 1.


In [39]:
constexpr size_t collatz_time(size_t n){
    size_t steps = 0;
    for(; n > 1; ++steps){
        n = (n % 2 == 0) ? n / 2 : 3 * n + 1;
    }
    return steps;
}

In [40]:
constexpr size_t sum_collatz_time(size_t n){
    size_t sum = 0;
    for(size_t i = 1; i <= n; ++i){
        sum += collatz_time(i);
    }
    return sum;
}

In [41]:
const auto s1 = sum_collatz_time(10000)

In [42]:
s1

(const unsigned long) 849666


In [43]:
const auto s2 = sum_collatz_time(100000)

In [44]:
s2

(const unsigned long) 10753840


In [45]:
const auto s3 = sum_collatz_time(1000000)

In [46]:
s3

(const unsigned long) 131434424


In [47]:
const auto s4 = sum_collatz_time(10000000)

In [48]:
s4

(const unsigned long) 1552724831


In [49]:
// const auto s5 = sum_collatz_time(10000000000)

Detection of UB:

In [50]:
// constexpr double v = 1/0;

In [52]:
constexpr int f(){
    std::vector<int> v(700);
    int* q = &v[7];
    v.resize(900);
    return *q;
}

In [54]:
// constexpr int x = f();

## 6.5 non-type template parameters